## Setup

In [12]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
import openai
import os
from prompts import PromptGenerator

In [3]:
openai.api_key = os.environ["OPENAI_API_KEY"]

## Test key

In [11]:
llm = ChatOpenAI(temperature=0.9, model_name="gpt-3.5-turbo")

In [10]:
answer = llm.invoke("Print a short joke")
print(answer)

content="Why don't scientists trust atoms?\n\nBecause they make up everything!" response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 11, 'total_tokens': 24}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-56b7d715-b0be-4901-8e6c-9eb6e5dba34c-0' usage_metadata={'input_tokens': 11, 'output_tokens': 13, 'total_tokens': 24}


## Output Parsing

In [15]:
# event_description = """MADSOC welcomes you to come along to our welcome drinks at Courtyard USYD to celebrate another year with MADSOC!! This is a great opportunity to come and meet our amazing exec in person, to ask any questions you may have about the society/classes/comp team, and to also make some new friends!

# WHEN: Thursday 2nd March, 5pm to 7pm

# WHERE: Courtyard USYD

# COST: |MADSOC members: FREE| |USU-members: $5| |Non-members: $10|"""


# price_check_prompt = PromptGenerator.fill_price_check_prompt(event_description = event_description)

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

In [16]:
category_list = ["Arts", "Academics", "Culture and Language", "Faith and Religion", "Political",
                 "Professional", "Social Impact and Advocacy", "Sport and Recreation"]

class event(BaseModel):
    category: str = Field(...,
                          description="The best category for the event",
                          enum=category_list)
    cost: int = Field(
        description="Minimum Cost of attending the event"
    )

In [17]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125").with_structured_output(
    event
)

chain = tagging_prompt | llm

In [18]:
inp = """MADSOC welcomes you to come along to our welcome drinks at Courtyard USYD to celebrate another year with MADSOC!! This is a great opportunity to come and meet our amazing exec in person, to ask any questions you may have about the society/classes/comp team, and to also make some new friends!

WHEN: Thursday 2nd March, 5pm to 7pm

WHERE: Courtyard USYD

COST: |MADSOC members: FREE| |USU-members: $5| |Non-members: $10|"""
chain.invoke({"input": inp})

event(category='Social Impact and Advocacy', cost=0)